# AWS EC2 Interactions w. Boto3 & Python

In [ ]:
import os, boto3

In [ ]:
# Retrieve the AWS credentials and region from environment variables
aws_access_key_id = os.getenv("AWS_ACCESS_KEY_ID")
aws_secret_access_key = os.getenv("AWS_SECRET_ACCESS_KEY")
region_name = os.getenv("AWS_DEFAULT_REGION")

print(f"AWS Access Key ID: {aws_access_key_id}")
print(f"AWS Secret Access Key: {aws_secret_access_key}")
print(f"Region: {region_name}")

ec2_instance_name = 'aws-ec2-interaction'

In [ ]:
ec2 = boto3.resource(
    "ec2",
    region_name=region_name,
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
)

In [ ]:
# store instance id
instance_id = None

In [ ]:
# check if instances exists
instances = ec2.instances.all()
instance_exists = False

for instance in instances:
    if instance.tags is not None:
        for tag in instance.tags:
            if tag["Key"] == "Name" and tag["Value"] == ec2_instance_name:
                instance_exists = True
                instance_id = instance.id
                print(f"An instance named {ec2_instance_name} with id {instance_id} already exists.")
                break
            
    if instance_exists:
        break

In [ ]:
# launch a new EC2 instance if it has not already been created
if not instance_exists:
    new_instance = ec2.create_instances(
        ImageId="ami-0b74f796d330ab49c",  # replace with a valid AMI_ID
        MinCount=1,
        MaxCount=1,
        InstanceType="t2.micro",
        KeyName="eu-central-aws-key-pair",
        TagSpecifications=[
            {
                "ResourceType": "instance",
                "Tags": [{"Key": "Name", "Value": ec2_instance_name}],
            }
        ],
    )
    instance_id = new_instance[0].id
    print(f"Instance named {ec2_instance_name} with id {instance_id} has been created.")

In [ ]:
# stop the instance
ec2.Instance(instance_id).stop()
print(f"Instance with id {instance_id} has been stopped.")

# check if the instance is stopped and retry until it is stopped
while True:
    instance = ec2.Instance(instance_id)
    if instance.state["Name"] == "stopped":
        print(f"Instance with id {instance_id} is stopped.")
        break

In [ ]:
# check if instance exists and if it is stopped
if instance_exists and instance.state["Name"] == "stopped":
    # start the instance
    ec2.Instance(instance_id).start()
    print(f"Instance with id {instance_id} has been started.")

    # check if the instance is running and retry until it is running
    while True:
        instance = ec2.Instance(instance_id)
        if instance.state["Name"] == "running":
            print(f"Instance with id {instance_id} is running.")
            break
else:
    print(f"Instance with id {instance_id} does not exist or is not stopped.")

In [ ]:
# check if instance exists and terminate it
if instance_exists:
    ec2.Instance(instance_id).terminate()
    print(f"Instance with id {instance_id} has been terminated.")
else:
    print(f"Instance with id {instance_id} does not exist.")